In [1]:
%store -r __import

In [2]:
__import

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm

In [4]:
train_df = pd.read_csv('/home/tuts/datasets/riid/train_raw_cleaned.csv', nrows=10_000_000)

In [5]:
train_df.head()

,Unnamed: 0,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,0,0,115,5692,0,1,3,1,NaN,NaN
1,1,1,56943,115,5716,0,2,2,1,37000.0,False
2,2,2,118363,115,128,0,0,0,1,55000.0,False
3,3,3,131167,115,7860,0,3,0,1,19000.0,False
4,4,4,137965,115,7922,0,4,1,1,11000.0,False


In [6]:
train_df.drop('Unnamed: 0', axis=1, inplace=True)

In [7]:
train_df.shape

(10000000, 10)

In [8]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000000 entries, 0 to 9999999
Data columns (total 10 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   row_id                          int64  
 1   timestamp                       int64  
 2   user_id                         int64  
 3   content_id                      int64  
 4   content_type_id                 int64  
 5   task_container_id               int64  
 6   user_answer                     int64  
 7   answered_correctly              int64  
 8   prior_question_elapsed_time     float64
 9   prior_question_had_explanation  object 
dtypes: float64(1), int64(8), object(1)
memory usage: 762.9+ MB


In [9]:
train_df.drop('row_id', inplace=True, axis=1)

In [20]:
target = 'answered_correctly'
train_df[target].value_counts()

1    6586860
0    3413140
Name: answered_correctly, dtype: int64

In [21]:
train_df.isna().sum()

timestamp                             0
user_id                               0
content_id                            0
content_type_id                       0
task_container_id                     0
user_answer                           0
answered_correctly                    0
prior_question_elapsed_time       40088
prior_question_had_explanation    40088
dtype: int64

In [22]:
train_df.isna().sum().sum()

80176

In [23]:
train_df.dropna(inplace=True)

In [24]:
train_df.user_id.nunique()

40182

In [25]:
train_df.content_id.nunique()

13500

In [26]:
train_df.content_type_id.nunique()

1

In [27]:
train_df.task_container_id.nunique()

10000

In [28]:
user_group = train_df.groupby(['user_id'])[target].median().reset_index()
user_group

,user_id,answered_correctly
0,115,1.0
1,124,0.0
2,2746,1.0
3,5382,1.0
4,8623,1.0
...,...,...
40177,220834282,1.0
40178,220839732,1.0
40179,220846142,0.0
40180,220849270,1.0


In [29]:
content_group = train_df.groupby(['content_id'])[target].median().reset_index()
content_group

,content_id,answered_correctly
0,0,1.0
1,1,1.0
2,2,1.0
3,3,1.0
4,4,1.0
...,...,...
13495,13518,1.0
13496,13519,1.0
13497,13520,1.0
13498,13521,1.0


In [30]:
task_group = train_df.groupby(['task_container_id'])[target].median().reset_index()
task_group

,task_container_id,answered_correctly
0,0,1.0
1,1,1.0
2,2,0.0
3,3,1.0
4,4,0.0
...,...,...
9995,9995,1.0
9996,9996,1.0
9997,9997,1.0
9998,9998,1.0


In [31]:
user_group.columns = ['user_id', 'user_performance']
content_group.columns = ['content_id', 'content_performance']
task_group.columns = ['task_container_id', 'task_performance']

In [32]:
def create_model(n_features, dropout, layers, lr):
    model = nn.Sequential(
        nn.Linear(n_features, layers[0]),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(layers[0], layers[1]),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(layers[1], layers[2]),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(layers[2], 1),
        nn.Sigmoid()
    )
    loss_fn = torch.nn.BCELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    return model, loss_fn, optimizer
    
batch_size = 128
n_features = len(train_df.columns)-1
dropout = 0.4
layers = [128,64, 32]
lr = 0.2

net, loss_fn, opt = create_model(n_features, dropout, layers, lr)

In [33]:
print(net)

Sequential(
  (0): Linear(in_features=8, out_features=128, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.4, inplace=False)
  (3): Linear(in_features=128, out_features=64, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.4, inplace=False)
  (6): Linear(in_features=64, out_features=32, bias=True)
  (7): ReLU()
  (8): Dropout(p=0.4, inplace=False)
  (9): Linear(in_features=32, out_features=1, bias=True)
  (10): Sigmoid()
)


In [34]:
x = torch.ones(8)
net.forward(x)

tensor([0.5417], grad_fn=<SigmoidBackward>)